In [ ]:
!pip install pyspark --quiet
!pip install -U -q PyDrive --quiet 
!apt install openjdk-8-jdk-headless &> /dev/null

     |████████████████████████████████| 281.4 MB 52 kB/s 
     |████████████████████████████████| 199 kB 71.7 MB/s 


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().set('spark.ui.port', '4050').setAppName("films").setMaster("local[2]")
sc = SparkContext.getOrCreate(conf=conf)

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip &> /dev/null
!unzip ngrok-stable-linux-amd64.zip &> /dev/null
get_ipython().system_raw('./ngrok http 4050 &')

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


In [ ]:
emission_type = ['Scope 1', 'Scope 2', 'Scope 3']

In [ ]:
emission_type_rdd = sc.parallelize(emission_type)
emission_type_rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [ ]:
emission_type_rdd.map(lambda emission_type: emission_type.title()).collect()

['Scope 1', 'Scope 2', 'Scope 3']

In [ ]:
emission_type_rdd.map(lambda emission_type: emission_type.title()).take(1)

['Scope 1']

In [ ]:
emission_type_rdd.map(lambda emission_type: emission_type.title())

PythonRDD[3] at RDD at PythonRDD.scala:53

In [ ]:
emission_type_rdd.map(lambda emission_type: emission_type.title()).filter(lambda emission_type: emission_type[0] == 'd')

PythonRDD[4] at RDD at PythonRDD.scala:53

In [ ]:
emission_type_rdd.filter(lambda emission_type: emission_type[0] == 'd').map(lambda emission_type: emission_type.title()).collect()

[]

In [ ]:
!pip install pyspark
!pip install findspark
import findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
findspark.init()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [ ]:
from pyspark import SparkContext, SparkConf

sc = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
data = sc.read.csv('WDIData_T.csv', inferSchema = True, header = True)
data.show(5)

+-----------+-----------+--------------------+--------------+----+-----------------+
|CountryName|CountryCode|       IndicatorName| IndicatorCode|Year|            Value|
+-----------+-----------+--------------------+--------------+----+-----------------+
| Arab World|        ARB|Adolescent fertil...|   SP.ADO.TFRT|1960| 134.772340547077|
| Arab World|        ARB|Age dependency ra...|   SP.POP.DPND|1960| 88.0611098786549|
| Arab World|        ARB|Age dependency ra...|SP.POP.DPND.OL|1960| 6.59195099288584|
| Arab World|        ARB|Age dependency ra...|SP.POP.DPND.YG|1960|81.32418639093659|
| Arab World|        ARB|Aquaculture produ...|ER.FSH.AQUA.MT|1960|           4600.0|
+-----------+-----------+--------------------+--------------+----+-----------------+
only showing top 5 rows



In [ ]:
data.count()

313386

In [ ]:
data.printSchema()

root
 |-- CountryName: string (nullable = true)
 |-- CountryCode: string (nullable = true)
 |-- IndicatorName: string (nullable = true)
 |-- IndicatorCode: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Value: double (nullable = true)



In [ ]:
def replace(column, value):
    return when(column!=value,column).otherwise(lit(None))

In [ ]:
from pyspark.sql.functions import when,lit,count,isnan,col
data.select([count(when(isnan(c)|col(c).isNull(),c)).alias(c) for c in data.columns]).show()

+-----------+-----------+-------------+-------------+----+-----+
|CountryName|CountryCode|IndicatorName|IndicatorCode|Year|Value|
+-----------+-----------+-------------+-------------+----+-----+
|          0|          0|            0|            1|   2|    1|
+-----------+-----------+-------------+-------------+----+-----+



In [ ]:
#deleting the column Market Category
data = data.drop("Market Category")
# deleting the all null values 
data = data.na.drop()

In [ ]:
print((data.count(), len(data.columns)))

(19083, 6)


In [ ]:
data.columns

['CountryName',
 'CountryCode',
 'IndicatorName',
 'IndicatorCode',
 'Year',
 'Value']

In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = ['CountryName',
 'CountryCode',
 'IndicatorName',
 'IndicatorCode',
 'Year',],
                           outputCol = "Input Attributes")

In [ ]:
from pyspark.ml.regression import RandomForestRegressor
regressor = RandomForestRegressor(labelCol = "MSRP",featuresCol = 'Input Attributes')

In [ ]:
regressor

RandomForestRegressor_580048ffda4f

In [ ]:
from pyspark.ml import Pipeline
pipeline  = Pipeline(stages = [assembler,regressor])

In [ ]:
#--Saving the Pipeline
pipeline.write().overwrite().save("pipeline_saved_model")

In [ ]:
pipelineModel = Pipeline.load('./pipeline_saved_model')

In [ ]:
data_train , data_test = data.randomSplit([0.8,0.2], seed = 123)

In [ ]:
data_train

DataFrame[CountryName: string, CountryCode: string, IndicatorName: string, IndicatorCode: string, Year: int, Value: double]

In [ ]:
try:
  %tensorflow_version 2.x 
except Exception:
  pass

!pip install tensorflow-io

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 26.9 MB 1.3 MB/s 
     |████████████████████████████████| 2.4 MB 59.6 MB/s 
  Attempting uninstall: tensorflow-io-gcs-filesystem
    Found existing installation: tensorflow-io-gcs-filesystem 0.27.0
    Uninstalling tensorflow-io-gcs-filesystem-0.27.0:
      Successfully uninstalled tensorflow-io-gcs-filesystem-0.27.0


In [ ]:
!npm install azurite@2.7.0

npm WARN deprecated request@2.87.0: request has been deprecated, see https://github.com/request/request/issues/3142
npm WARN deprecated uuid@3.4.0: Please upgrade  to version 7 or higher.  Older versions may use Math.random() in certain circumstances, which is known to be problematic.  See https://v8.dev/blog/math-random for details.
npm WARN deprecated har-validator@5.0.3: this library is no longer supported
npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ azurite@2.7.0
added 125 packages from 144 contributors and audited 125 packages in 7.765s

8 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run

In [ ]:
# The path for npm might not be exposed in PATH env,
# you can find it out through 'npm bin' command
npm_bin_path = get_ipython().getoutput('npm bin')[0]
print('npm bin path: ', npm_bin_path)

# Run `azurite-blob -s` as a background process. 
# IPython doesn't recognize `&` in inline bash cells.
get_ipython().system_raw(npm_bin_path + '/' + 'azurite-blob -s &')

npm bin path:  /content/node_modules/.bin


In [ ]:
import os
import tensorflow as tf
import tensorflow_io as tfio

# Switch to False to use Azure Storage instead:
use_emulator = True

if use_emulator:
  os.environ['TF_AZURE_USE_DEV_STORAGE'] = '1'
  account_name = 'devstoreaccount1'
else:
  # Replace <key> with Azure Storage Key, and <account> with Azure Storage Account
  os.environ['TF_AZURE_STORAGE_KEY'] = '<key>'
  account_name = 'divyanshpalia'


  # Alternatively, you can use a shared access signature (SAS) to authenticate with the Azure Storage Account
  os.environ['TF_AZURE_STORAGE_SAS'] = '<your sas>'
  account_name = 'divyanshpalia'

In [ ]:
pip install transport-co2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transport_co2 import Mode
list(Mode)

[<Mode.LIGHT_RAIL: (2184, 156)>,
 <Mode.SMALL_CAR: (168, 1.5)>,
 <Mode.LARGE_CAR: (220, 1.5)>,
 <Mode.SCOOTER: (86.4, 1.2)>,
 <Mode.BUS: (863, 12.7)>,
 <Mode.WALK: (0, 1)>,
 <Mode.CAR: (194, 1.5)>,
 <Mode.SUBWAY: (193.44, 31)>,
 <Mode.FERRY: (7425.6, 91)>,
 <Mode.TRANSIT: (1080.1466666666668, 66.56666666666666)>,
 <Mode.AIRPLANE: (25080, 88)>]

In [ ]:
Mode.SMALL_CAR.estimate_co2(distance_in_km=100)
# 11200.0

11200.0

In [ ]:
Mode.SMALL_CAR.estimate_co2(distance_in_km=100, occupancy=3)
# 5600.0

5600.0

In [ ]:
from transport_co2 import estimate_co2
estimate_co2(mode="light_rail", distance_in_km=100)
# 1400.0

1400.0

In [ ]:
estimate_co2(mode="light_rail", distance_in_km=100, occupancy=250)
# 873.6

873.6

In [ ]:
pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def get_data(self, api):
        response = requests.get(f"{api}")
        if response.status_code == 200:
            print("sucessfully fetched the data")
            self.formatted_print(response.json())
        else:
            print(f"Hello person, there's a {response.status_code} error with your request")

In [ ]:
def get_user_data(self, api, parameters):
        response = requests.get(f"{api}", params=parameters)
        if response.status_code == 200:
            print("sucessfully fetched the data with parameters provided")
            self.formatted_print(response.json())
        else:
            print(
                f"Hello person, there's a {response.status_code} error with your request")

In [ ]:
import requests
import json


class MakeApiCall:

    def get_data(self, api):
        response = requests.get(f"{api}")
        if response.status_code == 200:
            print("sucessfully fetched the data")
            self.formatted_print(response.json())
        else:
            print(
                f"Hello person, there's a {response.status_code} error with your request")

    def get_user_data(self, api, parameters):
        response = requests.get(f"{api}", params=parameters)
        if response.status_code == 200:
            print("sucessfully fetched the data with parameters provided")
            self.formatted_print(response.json())
        else:
            print(
                f"Hello person, there's a {response.status_code} error with your request")

    def formatted_print(self, obj):
        text = json.dumps(obj, sort_keys=True, indent=4)
        print(text)

    def __init__(self, api):
        # self.get_data(api)

        parameters = {
            "username": "kedark"
        }
        self.get_user_data(api, parameters)


if __name__ == "__main__":
    api_call = MakeApiCall("https://dev.to/api/articles")

sucessfully fetched the data with parameters provided
[
    {
        "canonical_url": "https://www.educative.io/edpresso/what-are-the-best-practices-while-using-bigquery",
        "collection_id": null,
        "comments_count": 0,
        "cover_image": "https://res.cloudinary.com/practicaldev/image/fetch/s--7eTmSNsF--/c_imagga_scale,f_auto,fl_progressive,h_420,q_auto,w_1000/https://dev-to-uploads.s3.amazonaws.com/uploads/articles/tncraaf09jupdiwpgs9e.png",
        "created_at": "2022-02-19T11:29:22Z",
        "crossposted_at": null,
        "description": "BigQuery is affordable and fast, but it can quickly consume a lot of processing power, increasing the...",
        "edited_at": null,
        "id": 994437,
        "last_comment_at": "2022-02-19T11:35:37Z",
        "path": "/kedark/what-are-the-best-practices-while-using-bigquery-3fcb",
        "positive_reactions_count": 11,
        "public_reactions_count": 11,
        "published_at": "2022-02-19T11:35:37Z",
        "published_t

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd 
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import FuncFormatter
from textwrap import wrap
%matplotlib inline



In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data = sc.read.csv('WDIData_T.csv', inferSchema = True, header = True)
data.show(5)


+-----------+-----------+--------------------+--------------+----+-----------------+
|CountryName|CountryCode|       IndicatorName| IndicatorCode|Year|            Value|
+-----------+-----------+--------------------+--------------+----+-----------------+
| Arab World|        ARB|Adolescent fertil...|   SP.ADO.TFRT|1960| 134.772340547077|
| Arab World|        ARB|Age dependency ra...|   SP.POP.DPND|1960| 88.0611098786549|
| Arab World|        ARB|Age dependency ra...|SP.POP.DPND.OL|1960| 6.59195099288584|
| Arab World|        ARB|Age dependency ra...|SP.POP.DPND.YG|1960|81.32418639093659|
| Arab World|        ARB|Aquaculture produ...|ER.FSH.AQUA.MT|1960|           4600.0|
+-----------+-----------+--------------------+--------------+----+-----------------+
only showing top 5 rows



In [ ]:
co2_indicator = 'CO2 emissions \(metric'
co2kt_indicator = 'CO2 emissions \(kt'
totGHG_indicator = 'Total greenhouse gas emissions \(kt of CO2'
hist_country = 'USA|GBR|FRA|CHN|JPN|DEU|IND|ITA|BRA|CAN'
totFuel='CO2 emissions from gaseous fuel consumption|CO2 emissions from liquid fuel consumption|CO2 emissions from solid fuel consumption'

In [ ]:

mask1 = data['IndicatorName'].str.contains(co2_indicator)
mask1kt = data['IndicatorName'].str.contains(co2kt_indicator)
mask1GHG = data['IndicatorName'].str.contains(totGHG_indicator)
mask2 = data['CountryCode'].str.contains(hist_country)
mask3 = data['Year'] == 1960
mask4 = data['Year'] == 1970
mask5 = data['Year'] == 2006
mask6 = data['Year'] == 2011
mask7 = data['Year'] == 2016
mask1Fuel = data['IndicatorName'].str.contains(totFuel)

In [ ]:
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),    
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),    
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199)] 

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials


In [ ]:
spreadsheet = 'https://docs.google.com/spreadsheets/d/1vd6hDT6k8lcWNMDYz13kqW7zXnIX5TVED9–4Kq7EAQnvI/edit#gid=0'
print("spreadhsheet : ",spreadsheet)
spreadsheet_key='spreadsheet_key=’1vd6hDT6k8lcWNMDYz13kqW7zXnIX5TVED9–4Kq7EAQnvI'
print("spreadsheet_key: ",spreadsheet_key)
for i in range(len(tableau20)):    
    r, g, b = tableau20[i]    
    tableau20[i] = (r / 255., g / 255., b / 255.) 

spreadhsheet :  https://docs.google.com/spreadsheets/d/1vd6hDT6k8lcWNMDYz13kqW7zXnIX5TVED9–4Kq7EAQnvI/edit#gid=0
spreadsheet_key:  spreadsheet_key=’1vd6hDT6k8lcWNMDYz13kqW7zXnIX5TVED9–4Kq7EAQnvI


In [ ]:
def thousands(x, pos):
    'The two args are the value and tick position'
    return '%1.1fK' % (x * 1e-4)

formatterK = FuncFormatter(thousands)

In [ ]:
def lineplot(data,countrylist,HorizPlots,PlotNo,Xmin,Xmax,Ymin,Ymax,Type):
    plt_vals=[]
    ax=plt.subplot(1, HorizPlots, PlotNo)
    ax.spines["top"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.get_yaxis().tick_left()
    if Type == 0:
        ax.spines["left"].set_visible(False)
        for y in range(5, 30, 5):
            plt.plot(range(1959, 2021), [y] * len(range(1959, 2021)), "--", lw=1.0, color="black", alpha=0.3)
        for i, col in enumerate(countrylist):
            plt_vals = data.loc[data['CountryCode'] == col]
            ax.plot(plt_vals['Year'].values, plt_vals['Value'].values, linewidth=1.5, color=tableau20[i])
            y_pos = plt_vals['Value'].values[-1] -0.3
            if plt_vals['CountryCode'].iloc[0] == 'USA':
                y_pos +=0.3
            elif plt_vals['CountryCode'].iloc[0] == 'JPN':
                y_pos +=0.3
            elif plt_vals['CountryCode'].iloc[0] == 'DEU':
                y_pos -=0.3
            elif plt_vals['CountryCode'].iloc[0] == 'BRA':
                y_pos +=0.3
            elif plt_vals['CountryCode'].iloc[0] == 'GBR':
                y_pos +=0.3
            plt.text(2016.5, y_pos, plt_vals['CountryName'].iloc[0], fontsize=12, color=tableau20[i])
        plt.title('CO2 Emissions (metric tons per capita)')
    
    if Type == 1:
        ax.set_xticks(ticks=[2006,2011,2016])
        ax.set_ylabel(data['IndicatorName'].iloc[0],fontsize=12)
        
        for i, col in enumerate(countrylist):
            plt_vals = data.loc[data['CountryCode'] == col]
            ax.plot(plt_vals['Year'].values, plt_vals['Value'].values, linewidth=1.5, color=tableau20[-1-i], marker ='o')
            y_pos = plt_vals['Value'].values[-1] -0.3
            x_pos = 2016.5
            if plt_vals['CountryCode'].iloc[0] == 'SAU':
                y_pos -=2.0
            elif plt_vals['CountryCode'].iloc[0] == 'TTO':
                y_pos -=1.0
            elif plt_vals['CountryCode'].iloc[0] == 'BHR':
                y_pos -=1.0
            elif plt_vals['CountryCode'].iloc[0] == 'ARE':
                y_pos +=1.0
            elif plt_vals['CountryCode'].iloc[0] == 'BRN':
                y_pos -=1.0
            elif plt_vals['CountryCode'].iloc[0] == 'GIB':
                x_pos +=4.5
                y_pos +=0.3
            plt.text(x_pos, y_pos, plt_vals['CountryName'].iloc[0], fontsize=12)
    
    if Type == 2:
        ax.set_yscale('log')
        ax.set_yticks(ticks=[1e3,1e5,1e7])
        ax.set_xticks(ticks=[2006,2011,2016])
        ax.set_ylabel(data['IndicatorName'].iloc[0],fontsize=12)
        ax.yaxis.set_major_formatter(formatterK)
        
        for i, col in enumerate(countrylist):
            plt_vals = data.loc[data['CountryCode'] == col]
            ax.plot(plt_vals['Year'].values, plt_vals['Value'].values, lw=1.5, marker='o', c=tableau20[i])
            y_pos = plt_vals['Value'].values[-1]
            if plt_vals['CountryCode'].iloc[0] == 'IDN':
                y_pos *=0.78
            elif plt_vals['CountryCode'].iloc[0] == 'SAU':
                y_pos *=0.90
            elif plt_vals['CountryCode'].iloc[0] == 'KOR':
                y_pos *=0.93
            elif plt_vals['CountryCode'].iloc[0] == 'DEU':
                y_pos *=1.05
            plt.text(2016.5, y_pos, plt_vals['CountryName'].iloc[0], fontsize=12)
    
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.axis([Xmin,Xmax,Ymin,Ymax])

In [ ]:
def scatterplot(data1,data2,countrylist1,countrylist2,HorizPlots,PlotNo,Xmin,Xmax,Ymin,Ymax,Type):
    i=0
    X = data1['Value']
    Y = data2['Value']
    X2 = data1['Value'][data1['CountryCode'].str.contains(countrylist1)]
    Y2 = data2['Value'][data2['CountryCode'].str.contains(countrylist1)]
    if Type == 1:
        X3 = data1['Value'][data1['CountryCode'].str.contains(countrylist2)]
        Y3 = data2['Value'][data2['CountryCode'].str.contains(countrylist2)]
    a = [i for i in range(0, int(Xmax))]
    ax = plt.subplot(1,HorizPlots,PlotNo) 
    if Type == 0:
        ax.scatter(X, Y, color=tableau20[1], edgecolor='black', s=70, marker='^')
        ax.scatter(X2, Y2, color=tableau20[-3], edgecolor='black', s=150)
    elif Type == 1:
        ax.scatter(X, Y, color=tableau20[1], edgecolor='black', s=70, marker='^')
        ax.scatter(X2, Y2, color=tableau20[-3], edgecolor='black', s=150)
        ax.scatter(X3, Y3, color=tableau20[-11], edgecolor='black', s=150)
    ax.plot(a, color='black', lw=0.5)
    ax.spines["top"].set_visible(False)    
    ax.spines["right"].set_visible(False)
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.tick_params(axis='both', which='minor', bottom=False, left=False)
    ax.set_xlabel(str(data1['Year'].iloc[0])+' '+data1['IndicatorName'].iloc[0],fontsize=12)
    ax.set_ylabel('2016 '+data2['IndicatorName'].iloc[0],fontsize=12)
    if Type == 0:
        for axis in [ax.xaxis, ax.yaxis]:
            axis.set_major_formatter(ScalarFormatter())
    elif Type == 1:
        ax.set_xticks(ticks=[1e3,1e5,1e7])
        ax.set_yticks(ticks=[1e3,1e5,1e7])
        for axis in [ax.xaxis, ax.yaxis]:
            axis.set_major_formatter(formatterK)
    
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.axis([Xmin,Xmax,Ymin,Ymax])

In [ ]:
def hBar(Val,Country,PlotNo,Title):
    X=[x for x, _ in enumerate(Val)]
    Y = np.array(Val)
    ax=plt.subplot(1, 5, PlotNo) 
    ax.spines["top"].set_visible(False)
    ax.spines["bottom"].set_visible(False)    
    ax.spines["right"].set_visible(False)
    for i, col in enumerate(Country):
        ax.text(29, i-0.1, str(int(round(Val.values[i])))+'%', fontsize=12)
    ax.barh(X,Y,color=tableau20[9])
    plt.title(("\n".join(wrap(Title, 15))))
    if PlotNo==1:
        plt.yticks(X, Country, wrap=True, fontsize=12)
    else:
        plt.yticks([])
    plt.xticks(fontsize=12)
    plt.xlim(0,60)

In [ ]:
countrylist = hist_country.split('|')
plt.figure(figsize=(10,7))

plt.show()

<Figure size 720x504 with 0 Axes>